# Exon-capture to Phylogeny Project

Calder Atta - FISH 546 Project

## Workflow (from Kuang et al. 2018)

1. Raw reads from Illumina sequencer
2. BCL to fastq format demultiplex (Illumina bcl2fastq package)
3. Remove adapter sequences and low quality score reads (Cutadapt 1.1 Trim_golare v0.2.8)
4. Remove the duplicates from PCR, parse the reads to each locus (A custom perl script: preads (supplementary))
5. Assemble the filtered reads into contigs (Trinity v20140717)
6. Merge the loci containing more than one contigs (Geneious v7.1.5)
7. Retrieve orthology by pairwise alignment to corresponding baits sequence (A custom Perl script: Smith-Waterman algorithm)
8. Identify orthology by comparing the retrieved sequence to the genome of O. nilotics (Blast v2.2.27)
9. Multiple sequences alignment (Clustal Omega v1.1.1)
10. Downstream analysis

## Notes from Kuang et al. 2018

- Samples and Genes
	- Sampled 43 species
	- 1 mt markers (COI)
	- 17817 nu markers (120bp baits)
		- target region <120bp was padded with T to 120bp
- Filtering
	- only used sequences found in all species and <5% missing data -> 570 markers
	- parameters for evaluating usefulness (calculated for all markers)
		1. Average pairwise difference (p-dist)
		2. Molecular clocklikeness (MCL)

## Getting started
Set short cuts for frequently used directories inside variables.

In [2]:
project = "/Users/calderatta/Desktop/FISH\ 546\ -\ Bioinformatics/project/"

In [3]:
biotools = "/Applications/bio-tools/"

Examine raw data.

In [9]:
ls {project}raw

TORN_Pool_10_S10_L006_R1_001.fastq* TORN_Pool_6_S6_L008_R1_001.fastq*
TORN_Pool_10_S10_L006_R2_001.fastq* TORN_Pool_6_S6_L008_R2_001.fastq*
TORN_Pool_10_S10_L008_R1_001.fastq* TORN_Pool_7_S7_L006_R1_001.fastq*
TORN_Pool_10_S10_L008_R2_001.fastq* TORN_Pool_7_S7_L006_R2_001.fastq*
TORN_Pool_4_S4_L006_R1_001.fastq*   TORN_Pool_7_S7_L008_R1_001.fastq*
TORN_Pool_4_S4_L006_R2_001.fastq*   TORN_Pool_7_S7_L008_R2_001.fastq*
TORN_Pool_4_S4_L008_R1_001.fastq*   TORN_Pool_8_S8_L006_R1_001.fastq*
TORN_Pool_4_S4_L008_R2_001.fastq*   TORN_Pool_8_S8_L006_R2_001.fastq*
TORN_Pool_5_S5_L006_R1_001.fastq*   TORN_Pool_8_S8_L008_R1_001.fastq*
TORN_Pool_5_S5_L006_R2_001.fastq*   TORN_Pool_8_S8_L008_R2_001.fastq*
TORN_Pool_5_S5_L008_R1_001.fastq*   TORN_Pool_9_S9_L006_R1_001.fastq*
TORN_Pool_5_S5_L008_R2_001.fastq*   TORN_Pool_9_S9_L006_R2_001.fastq*
TORN_Pool_6_S6_L006_R1_001.fastq*   TORN_Pool_9_S9_L008_R1_001.fastq*
TORN_Pool_6_S6_L006_R2_001.fastq*   TORN_Pool_9_S9_L008_R2_001.fastq*


Species 4 through 10 should be listed. For each species there are forward (R1) and reverse (R2) reads for each Illumina lane (L00#). Within each forward/reverse file pairs, the order of sequences is consistent.

In [10]:
!head {project}data/TORN_Pool_10_S10_L006_R1_001.fastq

@K00179:70:HHV7JBBXX:6:1101:24454:1209 1:N:0:AACGAAGT
TNTCTCTCTCTCTTGCTCTCTCTCTCTCTGTTTGAGCTCTCTCTCCCTCTCTCTCTCTCTCTGTCTCTCTCTGTTTGAGCTAACTCTCTCTCTCTGTTAGAGCTCTCTCTCGNTCGCTCTCTCTCGCTNTCGCTGGCTCGCACGCTCTCT
+
A#AAAFFFJJJFJFAJFJ-FFFAJFJ-FJ-AA<--7<-7FA7F<A--77A-7A-<777A-<F--A-<FF7F<-----777777F777A-77AF<---7A-----)-7-7-7)#--7--<)F<--7)-)#7A-77)--)7)))7)7)))--
@K00179:70:HHV7JBBXX:6:1101:26829:1209 1:N:0:AACGAAGT
CNCTTTCCTTCAGGAGAGACTCTGTCAGGAGGTGCAGGAGGAACAAAAGGAGCAAGAGGAGGAGGATCTGAAGGAGGGATGAGGTGTTGCAGGACGATGAACAGGAGGGGGAGCATGAGGAGGAGCAGGAGTAGGTGGAGCATAAGGAGG
+
A#AAFFJFFJJJJJJAFJJJAJFJ<JAAJJJJAJJJ7FFF<AJJFAJFFJ<AFJJF<AJJJFFJA77F7A--F-AAFF-7FJJJAF7FJ<AAFFJJ77AF7<<A<AJF))-))-)-77A<-<7AJF)<7)7-7-7<F-))7)---7--7)
@K00179:70:HHV7JBBXX:6:1101:4472:1226 1:N:0:AACGAAGT
GNAACAACATGGAGGTCAGAGGAGGAACAACATGGAGGTCAGAGGAGGAACAACATGGAGGTCAGAGGAGGAACAACATGGAGGTCAGAGAAGGCGCATCACGTATCTCAGANGAAAAGAAAGGAGGTNTGCAAAGACGAACGAGGGGGC


Now let's check that the number of sequences in R1 and R2 files match. First test on one file.

In [21]:
!grep -c "@" {project}data/TORN_Pool_10_S10_L008_R2_001.fastq

2419885


In [47]:
!grep -c '@' {project}data/*.fastq

/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_10_S10_L006_R1_001.fastq:2419885
/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_10_S10_L006_R2_001.fastq:2419885
/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_10_S10_L008_R1_001.fastq:1497568
/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_10_S10_L008_R2_001.fastq:1497568
/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_4_S4_L006_R1_001.fastq:3061218
/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_4_S4_L006_R2_001.fastq:3061218
/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_4_S4_L008_R1_001.fastq:2506641
/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_4_S4_L008_R2_001.fastq:2506641
/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/raw/TORN_Pool_5_S5_L006_R1_001.fastq:1248025
/Users/calderatta/Desktop/FISH 546 - Bioinforma

## 1. Raw Reads from Illumina Sequencer
This was already done.

## 2. BCL to fastq format demultiplex
This was already done.

## 3. Remove adapter sequences and low quality score reads

### FastQC - checking quality across all squences in each file

#### From DMG (Mac GUI application)
Open the application, and select file(s).

In [10]:
!open {biotools}FastQC.app

#### From Zip File (Windows or Mac)

In [6]:
!{biotools}FastQC/fastqc \
{project}data/*.fastq \
-o {project}analysis/fastqc

Started analysis of TORN_Pool_10_S10_L006_R1_001.fastq
Approx 5% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 10% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 15% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 20% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 25% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 30% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 35% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 40% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 45% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 50% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 55% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 60% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 65% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 70% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 75% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Approx 80% complete for TORN_Pool_10_S10_L006_R1_001.fastq
Ap

Approx 80% complete for TORN_Pool_4_S4_L008_R1_001.fastq
Approx 85% complete for TORN_Pool_4_S4_L008_R1_001.fastq
Approx 90% complete for TORN_Pool_4_S4_L008_R1_001.fastq
Approx 95% complete for TORN_Pool_4_S4_L008_R1_001.fastq
Analysis complete for TORN_Pool_4_S4_L008_R1_001.fastq
Started analysis of TORN_Pool_4_S4_L008_R2_001.fastq
Approx 5% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 10% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 15% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 20% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 25% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 30% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 35% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 40% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 45% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 50% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 55% complete for TORN_Pool_4_S4_L008_R2_001.fastq
Approx 60% complete for TORN_Pool_4_S4

Approx 70% complete for TORN_Pool_6_S6_L006_R2_001.fastq
Approx 75% complete for TORN_Pool_6_S6_L006_R2_001.fastq
Approx 80% complete for TORN_Pool_6_S6_L006_R2_001.fastq
Approx 85% complete for TORN_Pool_6_S6_L006_R2_001.fastq
Approx 90% complete for TORN_Pool_6_S6_L006_R2_001.fastq
Approx 95% complete for TORN_Pool_6_S6_L006_R2_001.fastq
Analysis complete for TORN_Pool_6_S6_L006_R2_001.fastq
Started analysis of TORN_Pool_6_S6_L008_R1_001.fastq
Approx 5% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 10% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 15% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 20% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 25% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 30% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 35% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 40% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 45% complete for TORN_Pool_6_S6_L008_R1_001.fastq
Approx 50% complete for TORN_Pool_6_S6

Approx 60% complete for TORN_Pool_8_S8_L006_R1_001.fastq
Approx 65% complete for TORN_Pool_8_S8_L006_R1_001.fastq
Approx 70% complete for TORN_Pool_8_S8_L006_R1_001.fastq
Approx 75% complete for TORN_Pool_8_S8_L006_R1_001.fastq
Approx 80% complete for TORN_Pool_8_S8_L006_R1_001.fastq
Approx 85% complete for TORN_Pool_8_S8_L006_R1_001.fastq
Approx 90% complete for TORN_Pool_8_S8_L006_R1_001.fastq
Approx 95% complete for TORN_Pool_8_S8_L006_R1_001.fastq
Analysis complete for TORN_Pool_8_S8_L006_R1_001.fastq
Started analysis of TORN_Pool_8_S8_L006_R2_001.fastq
Approx 5% complete for TORN_Pool_8_S8_L006_R2_001.fastq
Approx 10% complete for TORN_Pool_8_S8_L006_R2_001.fastq
Approx 15% complete for TORN_Pool_8_S8_L006_R2_001.fastq
Approx 20% complete for TORN_Pool_8_S8_L006_R2_001.fastq
Approx 25% complete for TORN_Pool_8_S8_L006_R2_001.fastq
Approx 30% complete for TORN_Pool_8_S8_L006_R2_001.fastq
Approx 35% complete for TORN_Pool_8_S8_L006_R2_001.fastq
Approx 40% complete for TORN_Pool_8_S8

Approx 50% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 55% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 60% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 65% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 70% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 75% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 80% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 85% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 90% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Approx 95% complete for TORN_Pool_9_S9_L008_R2_001.fastq
Analysis complete for TORN_Pool_9_S9_L008_R2_001.fastq


Produces an html file which you can open in a browser.

### Merge files from multiple lanes
Function: Merge the data on lane6 and lane8 together  
Input file: unziped fastq files(.fastq)  
Output file: merged files(.fastq)


Let's do a test run on one set of files.

In [39]:
!cat \
{project}data/TORN_Pool_4_S4_L006_R1_001.fastq \
{project}data/TORN_Pool_4_S4_L008_R1_001.fastq \
> {project}analysis/merge/TORN_Pool_4_S4_Merged_R1_001.fastq

Now let's try to do the rest using a loop (One for R1 and one of R2). For this I navigated to the data/ folder in Terminal and used the following lines of code. I couldn't figure out how to use absolute paths in Jupyter. The output files went into data/ but I moved them into analysis/merge/. New output file name convention is TORN_Pool_#_S#_R#.fastq.

In [ ]:
!(for i in *_L006_R1_001.fastq; do cat ${i%_L006_R1_001.fastq}_L006_R1_001.fastq ${i%_L006_R1_001.fastq}_L008_R1_001.fastq > ${i%_L006_R1_001.fastq}_R1.fastq; done)
!(for i in *_L006_R2_001.fastq; do cat ${i%_L006_R2_001.fastq}_L006_R2_001.fastq ${i%_L006_R2_001.fastq}_L008_R2_001.fastq > ${i%_L006_R2_001.fastq}_R2.fastq; done)

### Trim-Galore

Need to verify cutadapt and trimgalore are up-to-date.

In [4]:
!{biotools}cutadapt-master/cutadapt --version

1.18


In [5]:
!{biotools}TrimGalore-0.5.0/trim_galore --version


                        Quality-/Adapter-/RRBS-/Hard-Trimming
                                (powered by Cutadapt)
                                  version 0.5.0

                               Last update: 28 06 2018



Tried to run TrimGalore on one sample, but couldn't get to work. Need to try the for loop in source material.

In [35]:
!{biotools}TrimGalore-0.5.0/trim_galore \
{project}data/TORN_Pool_10_S10_L008_R2_001.fastq \
--path_to_cutadapt {biotools}cutadapt-master/cutadapt \
-o {project}analysis/trimgalore/

No quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)

Path to Cutadapt set as: '/Applications/bio-tools/cutadapt-master/cutadapt' (user defined)
1.18
Cutadapt seems to be working fine (tested command '/Applications/bio-tools/cutadapt-master/cutadapt --version')


AUTO-DETECTING ADAPTER TYPE
Attempting to auto-detect adapter type from the first 1 million sequences of the first file (>> /Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/data/TORN_Pool_10_S10_L008_R2_001.fastq <<)

Found perfect matches for the following adapter sequences:
Adapter type	Count	Sequence	Sequences analysed	Percentage
Illumina	62856	AGATCGGAAGAGC	1000000	6.29
Nextera	6	CTGTCTCTTATA	1000000	0.00
smallRNA	1	TGGAATTCTCGG	1000000	0.00
Using Illumina adapter for trimming (count: 62856). Second best hit was Nextera (count: 6)

Writing report to '/Users/calderatta/Desktop/FISH 546 - Bioinformatics/project/analysis/trimgalore/TORN_Pool_10_S10_L008_R2_00

Function: Trim the adapter and low quality reads in fastq file  
Input file: merged fastq files(.fastq)  
Output file: trimmed files(.fq)  

As with merge, I also navigated to the analysis/merge/ folder in Terminal and used the following line of code. The output files went into analysis/merge/ but I moved them into analysis/trimgalore/. New output file name convention is TORN_Pool_#_S#_R#.fastq.

In [ ]:
(for i in *_R1.fastq; do /Applications/bio-tools/TrimGalore-0.5.0/trim_galore --path_to_cutadapt /Applications/bio-tools/cutadapt-master/cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC -a2 AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT --paired ${i%_R1.fastq}_R1.fastq ${i%_R1.fastq}_R2.fastq; done) >& trim.log.txt